# ရိုးရှင်းသော ပုံရိပ် ခွဲခြားစနစ်

ဒီ notebook က သင့်ကို အကြိုပြင်ဆင်ထားသော နာရူရယ်ကွန်ယက်ကို အသုံးပြုပြီး ပုံရိပ်များကို ခွဲခြားပုံပြောပြပေးပါမယ်။

**သင်လေ့လာနိုင်မယ့်အရာများ:**
- အကြိုပြင်ဆင်ထားသော မော်ဒယ်ကို ဘယ်လို load လုပ်ပြီး အသုံးပြုမလဲ
- ပုံရိပ်များကို ကြိုတင်ပြင်ဆင်ခြင်း
- ပုံရိပ်များအပေါ် အတိအကျခန့်မှန်းချက်များ ပြုလုပ်ခြင်း
- ယုံကြည်မှုအဆင့်များကို နားလည်ခြင်း

**အသုံးပြုမှုကိစ္စ:** ပုံရိပ်များအတွင်းရှိ အရာဝတ္ထုများ (ဥပမာ "ကြောင်", "ခွေး", "ကား" စသည်) ကို ဖော်ထုတ်ခြင်း


## အဆင့် ၁ - လိုအပ်သော စာကြည့်တိုက်များကို တင်သွင်းပါ

လိုအပ်သော ကိရိယာများကို တင်သွင်းကြမယ်။ အားလုံးကို မနားလည်သေးရင် စိတ်မပူပါနဲ့!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## အဆင့် ၂ - ကြိုတင်လေ့ကျင့်ထားသော မော်ဒယ်ကို Load လုပ်ပါ

ကျွန်တော်တို့ **MobileNetV2** ကို အသုံးပြုပါမယ်၊ ဒါဟာ သန်းပေါင်းများစွာသော ပုံများပေါ်မှာ ရှိပြီးသား Neural Network တစ်ခုဖြစ်ပါတယ်။

ဒါကို **Transfer Learning** လို့ခေါ်ပါတယ် - တစ်ခြားသူလေ့ကျင့်ထားတဲ့ မော်ဒယ်ကို အသုံးပြုတာပါ!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## အဆင့် ၃ - အကူအညီပေးသော Function များ

အကြောင်းအရာများကို မော်ဒယ်အတွက် load လုပ်ပြီး ပြင်ဆင်ရန် Function များကို ဖန်တီးကြမယ်။


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## အဆင့် ၄ - နမူနာပုံများကို စမ်းသပ်ပါ

အင်တာနက်မှ ပုံများကို သတ်မှတ်ကြည့်ရအောင်!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ရုပ်ပုံတစ်ခုချင်းစီကို အမျိုးအစားခွဲပါ


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## အဆင့် ၅: သင့်ပုံများကို စမ်းကြည့်ပါ!

အောက်ပါ URL ကို သင့်ရဲ့ အမျိုးအစားသတ်မှတ်လိုသော ပုံ URL တစ်ခုဖြင့် အစားထိုးပါ။


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ဘာတွေဖြစ်ခဲ့တာလဲ?

1. **Pre-trained model ကို load လုပ်ခဲ့တယ်** - MobileNetV2 ကို သန်းပေါင်းများစွာသော ပုံများပေါ်မှာ training လုပ်ထားပါတယ်။
2. **ပုံတွေကို preprocess လုပ်ခဲ့တယ်** - Model အတွက် အရွယ်အစားကို ပြင်ဆင်ပြီး format လုပ်ပေးခဲ့တယ်။
3. **Model က ခန့်မှန်းချက်တွေ ပြုလုပ်ခဲ့တယ်** - 1000 object class အတွက် probability တွေကို output ထုတ်ပေးခဲ့တယ်။
4. **ရလဒ်တွေကို decode လုပ်ခဲ့တယ်** - နံပါတ်တွေကို လူနားလည်နိုင်တဲ့ label တွေကို ပြောင်းပေးခဲ့တယ်။

### Confidence Score ကို နားလည်ခြင်း

- **90-100%**: အရမ်းယုံကြည်မှုရှိ (မှန်မယ်လို့ အတော်လေးသေချာ)
- **70-90%**: ယုံကြည်မှုရှိ (မှန်နိုင်တယ်)
- **50-70%**: တစ်ချို့ယုံကြည်မှုရှိ (မှန်နိုင်တယ်)
- **50% အောက်**: ယုံကြည်မှုမရှိသလောက် (မသေချာ)

### ဘာကြောင့် ခန့်မှန်းချက်တွေ မှားနိုင်သလဲ?

- **Angle သို့မဟုတ် အလင်းအရောင် မမှန်ခြင်း** - Model ကို ပုံမှန်ဓာတ်ပုံတွေမှာ training လုပ်ထားတာပါ။
- **Object အများကြီးပါဝင်ခြင်း** - Model က အဓိက object တစ်ခုကိုသာ မျှော်လင့်ထားပါတယ်။
- **ရှားပါး object တွေ** - Model က category 1000 ခုသာ သိပါတယ်။
- **ပုံအရည်အသွေး မကောင်းခြင်း** - မိုဃ်းမဲတဲ့ပုံတွေ သို့မဟုတ် pixelated ပုံတွေကို ခန့်မှန်းဖို့ ပိုခက်ပါတယ်။

---


## 🚀 နောက်တစ်ဆင့်လုပ်ဆောင်ရန်

1. **ပုံအမျိုးမျိုးကို စမ်းကြည့်ပါ:**
   - [Unsplash](https://unsplash.com) မှ ပုံများရှာဖွေပါ
   - Right-click → "Copy image address" ကိုနှိပ်ပြီး URL ကိုရယူပါ

2. **စမ်းသပ်ကြည့်ပါ:**
   - အ抽象ပန်းချီပုံများတွင် ဘာဖြစ်မလဲ?
   - အမြင်အမျိုးမျိုးမှ အရာဝတ္ထုများကို မှတ်မိနိုင်ပါသလား?
   - အရာဝတ္ထုများစွာကို တစ်ခါတည်း တွေ့ရှိနိုင်ပါသလား?

3. **ပိုမိုလေ့လာပါ:**
   - [Computer Vision သင်ခန်းစာများ](../lessons/4-ComputerVision/README.md) ကို စူးစမ်းပါ
   - သင့်ကိုယ်ပိုင် ပုံခွဲခြားစနစ်ကို လေ့ကျင့်ပေးပါ
   - CNNs (Convolutional Neural Networks) အလုပ်လုပ်ပုံကို နားလည်ပါ

---

## 🎉 ဂုဏ်ယူပါတယ်!

သင်သည် နောက်ဆုံးပေါ် နယူးရယ်နက်ဝက်ကို အသုံးပြု၍ ပုံခွဲခြားစနစ်တစ်ခုကို တည်ဆောက်ပြီးပါပြီ!

ဒီနည်းပညာတစ်ခုသည် အောက်ပါအရာများကို အားပေးပါသည်။
- Google Photos (သင့်ရဲ့ပုံများကို စီစဉ်ပေးခြင်း)
- ကိုယ်တိုင်မောင်းနှင်သော ကားများ (အရာဝတ္ထုများကို မှတ်မိခြင်း)
- ဆေးဘက်ဆိုင်ရာ ရောဂါရှာဖွေခြင်း (X-ray များကို ခွဲခြားစစ်ဆေးခြင်း)
- အရည်အသွေးထိန်းချုပ်မှု (အနာအဆာများကို ရှာဖွေခြင်း)

ဆက်လက် စူးစမ်းလေ့လာပါ! 🚀



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
